In [1]:
import pandas as pd
from PIL import Image, ImageFilter

In [2]:

#Read image
im = Image.open('data/images/20171118200710_1.jpg')
#Display image
im.show()

#Applying a filter to the image
im_sharp = im.filter( ImageFilter.SHARPEN )
#Saving the filtered image to a new file
im_sharp.save( 'image_sharpened.jpg', 'JPEG' )

#Splitting the image into its respective bands, i.e. Red, Green,
#and Blue for RGB
r,g,b = im_sharp.split()

#Viewing EXIF data embedded in image
#exif_data = im._getexif()
#exif_data

In [3]:
w, h = yourImage.size
yourImage.crop((0, 30, w, h-30)).save(...)

NameError: name 'yourImage' is not defined

In [68]:

img = Image.open("data/images/20171118200710_1.jpg")
img = im.filter( ImageFilter.SHARPEN )
w, h = img.size
p1_area = (570, 205, w-300, h-820)
p2_area = (570, 230, w-300, h-795)
p3_area = (570, 253, w-300, h-770)
p4_area = (570, 275, w-300, h-750)
p5_area = (570, 303, w-300, h-725)
p6_area = (570, 328, w-300, h-700)
p7_area = (570, 350, w-300, h-678)
p8_area = (570, 373, w-300, h-655)
p9_area = (570, 403, w-300, h-628)
p10_area = (570, 425, w-300, h-605)
p11_area = (570, 448, w-300, h-580)
p12_area = (570, 470, w-300, h-560)
cropped_img = img.crop(p12_area)
cropped_img.show()

In [72]:
def split_players(img=None, sharpen=True, save_path="images_processed/"):
    """Takes an MWO team score image and splits it into 12 rectanges, 1 for each player's stats."""
    if img is not None:
        #load image
        img = Image.open(img)
        if sharpen:
            #sharpen image
            img = im.filter(ImageFilter.SHARPEN)
        #set areas for player rectangles
        p1_area = (570, 205, w-300, h-820)
        p2_area = (570, 230, w-300, h-795)
        p3_area = (570, 253, w-300, h-770)
        p4_area = (570, 275, w-300, h-750)
        p5_area = (570, 303, w-300, h-725)
        p6_area = (570, 328, w-300, h-700)
        p7_area = (570, 350, w-300, h-678)
        p8_area = (570, 373, w-300, h-655)
        p9_area = (570, 403, w-300, h-628)
        p10_area = (570, 425, w-300, h-605)
        p11_area = (570, 448, w-300, h-580)
        p12_area = (570, 470, w-300, h-560)
        #split into player images
        p1 = img.crop(p1_area)
        p2 = img.crop(p2_area)
        p3 = img.crop(p3_area)
        p4 = img.crop(p4_area)
        p5 = img.crop(p5_area)
        p6 = img.crop(p6_area)
        p7 = img.crop(p7_area)
        p8 = img.crop(p8_area)
        p9 = img.crop(p9_area)
        p10 = img.crop(p10_area)
        p11 = img.crop(p11_area)
        p12 = img.crop(p12_area)
        player_images = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12]
    #return array of player images
    else:
        print("No image selected")
    return player_images

In [73]:
player_imgs = split_players("data/images/20171118200710_1.jpg")

In [75]:
for img in player_imgs:
    img.show()
